# Tutoriel pytorch - TP3 - IFT725

Tel que mentionné dans l'énoncé du travail, vous devez recopier les blocs de code du tutoriel suivant

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

en donnant, pour chaque bloc, une description en format "markdown" de son contenu.

# Warm-up: numpy
## Fonctionnement global du code

Entrainement d'un modèle de régression polynomial simple de degré 3 avec squared error loss.
Les gradients sont calculés manuellement et la mise à jour des poids est fait par descente de
gradient stochastique. L'ensemble d'entraînement est constitué de 2000 points générés à partir de la fonction sinus.
On souhaite donc que le modèle obtenu s'apparente à la vraie fonction sinus.

## Principales variables

In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 3536.3034711900973
199 2341.669624772834
299 1551.6298857621118
399 1029.153476982161
499 683.6216507035274
599 455.1073942762032
699 303.980175854083
799 204.03162244272298
899 137.92954921168533
999 94.21170635722574
1099 65.2977315579286
1199 46.17443850846054
1299 33.52638261411303
1399 25.16089002370266
1499 19.627817386812403
1599 15.96808921310856
1699 13.547397844594965
1799 11.946223163320559
1899 10.887097937912763
1999 10.186504829216272
Result: y = 0.003189451373131106 + 0.8208886569061805 x + -0.0005502334345650464 x^2 + -0.08823072776184114 x^3


# PyTorch: Tensors
## Fonctionnement global du code

Entrainement d'un modèle de régression polynomial simple de degré 3 avec squared error loss comme à l'étape
précédente. Toutefois, ici les données et les poids du modèles sont plutôt conservés dans des tenseurs PyTorch.

## Principales variables

In [4]:
# -*- coding: utf-8 -*-

import torch
import math


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 484.68170166015625
199 345.527099609375
299 247.06814575195312
399 177.40106201171875
499 128.1061248779297
599 93.22583770751953
699 68.54486083984375
799 51.08077621459961
899 38.7232780456543
999 29.979068756103516
1099 23.791641235351562
1199 19.413358688354492
1299 16.31524085998535
1399 14.122967720031738
1499 12.571677207946777
1599 11.473958969116211
1699 10.697185516357422
1799 10.147527694702148
1899 9.758570671081543
1999 9.483333587646484
Result: y = 0.02731378562748432 + 0.8572689890861511 x + -0.004712081514298916 x^2 + -0.09340550750494003 x^3


# PyTorch: Tensors and autograd
## Fonctionnement global du code

Entrainement d'un modèle de régression polynomial simple de degré 3 avec squared error loss comme à
l'étape précédente. Toutefois, en plus d'avoir les données et les poids du modèles dans des tenseurs PyTorch,
les gradients utilisés pour la backward pass sont automatiquement calculés par autograd.
Ils sont ensuite mis à None pour éviter l'accumulation du gradient.


## Principales variables



In [5]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 1857.18701171875
199 1231.3262939453125
299 817.3952026367188
399 543.626708984375
499 362.55706787109375
599 242.79653930664062
699 163.58456420898438
799 111.19156646728516
899 76.53660583496094
999 53.61391067504883
1099 38.451255798339844
1199 28.421388626098633
1299 21.786592483520508
1399 17.39756965637207
1499 14.494036674499512
1599 12.573155403137207
1699 11.302360534667969
1799 10.461570739746094
1899 9.905294418334961
1999 9.537238121032715
Result: y = 0.003041293239220977 + 0.8308057188987732 x + -0.000524673261679709 x^2 + -0.08964134007692337 x^3


# PyTorch: Defining new autograd function
## Fonctionnement global du code

Entrainement d'un modèle de régression polynomial incluant le polynômne de Legendre.
Une sous-classe d'autograd.Function est mise en place pour définir la forward pass et la backward pass associées au
polynôme de Legendre. De cette façon, l'utilisation du polynôme peut-être intégré dans le calcul de la prédiction et
son gradient propagé correctement lors de la rétroprogration de la loss.


## Principales variables

In [6]:
# -*- coding: utf-8 -*-
import torch
import math


class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

99 209.9583282470703
199 144.6602020263672
299 100.7025146484375
399 71.03520965576172
499 50.97850799560547
599 37.40315246582031
699 28.20688247680664
799 21.97319221496582
899 17.7457275390625
999 14.877889633178711
1099 12.93176555633545
1199 11.610918998718262
1299 10.71424674987793
1399 10.105476379394531
1499 9.69210433959961
1599 9.411375045776367
1699 9.220744132995605
1799 9.091285705566406
1899 9.003360748291016
1999 8.943641662597656
Result: y = 3.5881797533221516e-09 + -2.208526849746704 * P3(-1.6777875755380478e-09 + 0.2554861009120941 x)


# PyTorch: nn
## Fonctionnement global du code

Entrainement d'un modèle de régression polynomial de degré 3 avec squared error. L'implémentation du modèle est facilité par
des modules de la librairie nn. Particulièrement, avec l'aide du Sequential nous définissons une composition de
fonctions sous forme de "layers". Ici nous exécutons un produit scalaire (fully-connected layer avec 3 inputs + 1 biais
et 1 output) et effectuons un flatten sur le tenseur de sortie. Avant d'effectuer la rétroprogation, nous nous assurons
que les gradients sont bien à zéro pour éviter l'accumulation du gradient. Suite à la rétropropagation les poids sont
mis à jour avec une descente de gradient stochastique.


## Principales variables

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

# PyTorch: optim
## Fonctionnement global du code

Ici on effectue la même procédure qu'à l'étape précédente. Toutefois, au lieu d'effectuer la mise à jour des poids
manuellement avec la descente de gradients stochastique, nous laissons le travail à un optimiseur PyTorch. Ici,
l'optimiseur utilise l'algorithme RMSprop et la mise à jour des poids est effectué via l'appel .step().


## Principales variables



In [ ]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use RMSprop; the optim package contains many other
# optimization algorithms. The first argument to the RMSprop constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')





# PyTorch : Custom nn Modules
## Fonctionnement global du code

Toujours avec pour but d'entraîner une régression polynomial de degré 3 avec squared error loss, nous définissons une classe hériant de nn.Module pour contenir notre modèle plutôt que de storer celui-ci dans une variable contenant un Sequential. Nous devons fournir à PyTorch les paramètres du modèle (nn.Parameter) pour qu'il garde en mémoire les gradients requis lors de l'entraînement. Ne devons également définir la forward pass. Notons ici que l'entraînement du modèle est ensuite effectué à l'aide d'une descente de gradient stochastique avec momentum.


## Principales variables

In [2]:
# -*- coding: utf-8 -*-
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the nn.Linear
# module which is members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 1307.0467529296875
199 882.2890014648438
299 597.0279541015625
399 405.29217529296875
499 276.30841064453125
599 189.4618682861328
699 130.93365478515625
799 91.45258331298828
899 64.79441833496094
999 46.7765998840332
1099 34.58622741699219
1199 26.32990837097168
1299 20.73217010498047
1399 16.93280601501465
1499 14.351251602172852
1599 12.595186233520508
1699 11.399299621582031
1799 10.584009170532227
1899 10.02751350402832
1999 9.647246360778809
Result: y = 0.022542858496308327 + 0.8378786444664001 x + -0.0038890186697244644 x^2 + -0.09064740687608719 x^3


# PyTorch : Custom nn Modules
## Fonctionnement global du code

Dans la section suivante, nous définissons un modèle de régression polynomiale de degré variant de 4 à 5.
Le coefficient associé au 4e et 5e degré est partagé. Toutefois, lors de la forward pass, l'apparition du 5e degré est déterminé par un processus aléatoire. Le but de l'exercice est de montrer que le graph de calcul utilisé par PyTorch dans la rétropropagation est dynamique et peut donc s'adapter sans problème à la forward pass effectué.
Dans le cas présent, le gradient dL/de sera calculé en fonction de la présence ou l'absence du 5e degré.


## Principales variables

In [3]:
# -*- coding: utf-8 -*-
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 121.32887268066406
3999 64.0608901977539
5999 33.49326705932617
7999 19.777589797973633
9999 13.801015853881836
11999 11.070599555969238
13999 9.338671684265137
15999 9.376082420349121
17999 9.089390754699707
19999 8.629243850708008
21999 8.917704582214355
23999 8.529715538024902
25999 8.536824226379395
27999 8.87140941619873
29999 8.615251541137695
Result: y = 0.000767931982409209 + 0.8588058352470398 x + -0.0006282383110374212 x^2 + -0.09385582059621811 x^3 + 9.319775563199073e-05 x^4 ? + 9.319775563199073e-05 x^5 ?
